In [26]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

standard = StandardScaler()
minmax = MinMaxScaler()
import pandas as pd
import pickle
import gym
import matplotlib.pyplot as plt


dataset = "./train_FD001.txt"
hierarchicalPath= "./hierarchical_100_unit.pkl"
output_dir= './results/v-20'

In [2]:

df_full = pd.read_csv( dataset, sep=" ", header=None, skipinitialspace=True).dropna(axis=1)
df_full = df_full.rename(columns={0: 'unit', 1: 'cycle', 2: 'W1', 3: 'W2', 4: 'W3'})
# Filter df_full for engine units 1 to 5
# df_full = df_full[df_full['unit'].isin(range(1,26))].reset_index(drop=True)
df_full = df_full.reset_index(drop=True)

df = df_full
df_full.columns = df_full.columns.astype(str)

df_A = df[df.columns[[0, 1]]] # unit and cycle
df_W = df[df.columns[[2, 3, 4]]] # W1 w2 w3 (opration conditions)
df_S = df[df.columns[list(range(5, 26))]] #sensors
df_X = pd.concat([df_W, df_S], axis=1)

df[['W1', 'W2', 'W3', *map(str, range(5, 26))]] = minmax.fit_transform(pd.concat([df_W, df_S], axis=1))

In [3]:

cols_to_drop = df.nunique()[df.nunique() == 1].index
df = df.drop(cols_to_drop, axis=1)

cols_to_drop = df.nunique()[df.nunique() == 2].index
df = df.drop(cols_to_drop, axis=1)

In [ ]:

with open(hierarchicalPath, 'rb') as f:
    hierarchical = pickle.load(f)

print(hierarchical)


In [ ]:
# Convert strings to integers if needed
hierarchical = [[int(x) for x in arr] for arr in hierarchical]

# Now compute the differences
differences = [arr[-1] - arr[0] for arr in hierarchical]
print(differences)

# Plotting
sorted_differences = sorted(differences)
plt.figure(figsize=(10, 5))
plt.bar(range(len(sorted_differences)), sorted_differences, color='skyblue')
plt.title('Bar Chart of Sorted Index Differences (Last - First)')
plt.xlabel('Sorted Array Index')
plt.ylabel('Index Difference')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:

# --- نمودار فراوانی (هیستوگرام) ---
plt.figure(figsize=(10, 5))
plt.hist(differences, bins=20, color='orange', edgecolor='black')
plt.title('Histogram of Index Differences')
plt.xlabel('Index Difference (Last - First)')
plt.ylabel('Frequency')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

<hr/>
RL
<hr/>


In [ ]:
df_X[['W1', 'W2', 'W3', *map(str, range(5, 26))]] = minmax.fit_transform(pd.concat([df_X], axis=1))
cols_to_drop = df_X.nunique()[df_X.nunique() == 1].index
df_X = df_X.drop(cols_to_drop, axis=1)
df_X
# df_hmm

In [110]:
c_f = -30
c_r = -10
do_nothing = 0
policy = {}
policy_test = {}
engine_unit = 1


In [111]:
import math
class CustomEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, is_training=True, verbose=False, nn='dnn', input_data='raw', srla=True):
        self.train = is_training
        self.verbose = verbose
        self.nn = nn
        self.input_data = input_data
        self.srla = srla
        
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(df_X.shape[1],))

        self.action_space = gym.spaces.Discrete(2)
        self.reward = 0
        self.done = False
        self.engine_unit = engine_unit
        self.cycle = int(hierarchical[self.engine_unit - 1][0]) - 1
        self.engine_df_A = df_A[df_A['unit'] == self.engine_unit]
         
        self.X = df_X[self.engine_df_A.index[0]:self.engine_df_A.index[-1] + 1]
 
       
        self.state =  self.X.iloc[self.cycle]
        self.failure_state = self.engine_df_A['cycle'].max() - 1

    def get_next_engine_data(self ,x_engine_unit):
        self.engine_unit = x_engine_unit


        self.engine_df_A = df_A[df_A['unit'] == self.engine_unit]
        self.X = df_X[self.engine_df_A.index[0]:self.engine_df_A.index[-1] + 1]

        self.failure_state = self.engine_df_A['cycle'].max() - 1
        return self.X

    def step(self, action):
        if action == 0:
            if self.verbose:
                print("|hold|:", self.cycle)
            if self.cycle == self.failure_state:
                fraction = 1 / (int(hierarchical[self.engine_unit - 1][-1]) - int(hierarchical[self.engine_unit - 1][0]))
                self.reward = (c_r + c_f) /  math.exp(fraction)
               
                self.state =  self.X.iloc[self.cycle]
                self.done = True

                if self.train:
                    policy[self.engine_unit] = {'unit': self.engine_unit,
                                                'failure_state': self.failure_state,
                                                'replace_state': self.cycle}
                else:
                    policy_test[self.engine_unit] = {'unit': self.engine_unit,
                                                     'failure_state': self.failure_state,
                                                     'replace_state': self.cycle,
                                                     'reward': self.reward}
                if self.verbose:
                    print("|cycle reached failure state|:", self.cycle, "reward:", self.reward, '\n')
            else:
                self.reward = do_nothing
                self.cycle += 1
                if self.srla:

                    if self.cycle in hierarchical[self.engine_unit - 1]:
                  
                        self.state =  self.X.iloc[self.cycle]
                        self.done = False
                        if self.verbose:
                            print("Danger zone!",'\n')
                            print("|system running|", "reward:", self.reward, '\n')
                    else:
                        # print('*****pass srla*****')
                        pass
                else:
                   
                    self.state =  self.X.iloc[self.cycle]
                    self.done = False
                    if self.verbose:
                        print("|system running|", "reward:", self.reward, '\n')
        elif action == 1:
            if self.verbose:
                print("|replace|:", self.cycle)
            if self.cycle == self.failure_state:
                fraction = 1 / (int(hierarchical[self.engine_unit - 1][-1]) - int(hierarchical[self.engine_unit - 1][0]))
                self.reward = (c_r + c_f) /  math.exp(fraction)
                # self.reward = (c_r + c_f) / (int(hierarchical[self.engine_unit - 1][-1]) - int(hierarchical[self.engine_unit - 1][0]))
            else:

                self.reward = c_r * (int(hierarchical[self.engine_unit - 1][-1]) - self.cycle) /( int(hierarchical[self.engine_unit - 1][-1]) - int(hierarchical[self.engine_unit - 1][0]))
 
            self.state =  self.X.iloc[self.cycle]
            if self.train:
                policy[self.engine_unit] = {'unit': self.engine_unit,
                                            'failure_state': self.failure_state,
                                            'replace_state': self.cycle}
            else:
                policy_test[self.engine_unit] = {'unit': self.engine_unit,
                                                 'failure_state': self.failure_state,
                                                 'replace_state': self.cycle,
                                                 'reward': self.reward}
                print(policy_test[self.engine_unit])
            self.done = True
        if self.verbose:
            print("reward:", self.reward, '\n')
            print("******************************************************************")
        info = {}
        return self.state, self.reward, self.done, info

    def reset(self , engine_unit):
        self.X = self.get_next_engine_data(engine_unit)
        # self.cycle = 166
        self.cycle = int(hierarchical[self.engine_unit - 1][0]) - 1
  
        self.state =  self.X.iloc[self.cycle]
        self.done = False
        return self.state

In [112]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import gym


class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, action_dim)
        )

    def forward(self, x):
        return self.net(x)


class DQNAgent:
    def __init__(self, state_dim, action_dim, gamma=0.99, lr=1e-4,
                 batch_size=256, buffer_size=5000, epsilon_start=1.0,
                 epsilon_end=0, epsilon_decay=0.995, target_update=240):

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.batch_size = batch_size
        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.target_update = target_update
        self.step_count = 0
        self.losses = []


        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.policy_net = QNetwork(state_dim, action_dim).to(self.device)
        self.target_net = QNetwork(state_dim, action_dim).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=lr)
        self.criterion = nn.MSELoss()

        self.memory = deque(maxlen=buffer_size)

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            return random.randrange(self.action_dim)
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        with torch.no_grad():
            q_values = self.policy_net(state)
        return torch.argmax(q_values).item()

    def store_transition(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def train_step(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).unsqueeze(1).to(self.device)
        rewards = torch.FloatTensor(rewards).unsqueeze(1).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).unsqueeze(1).to(self.device)

        q_values = self.policy_net(states).gather(1, actions)
        with torch.no_grad():
            max_next_q_values = self.target_net(next_states).max(1, keepdim=True)[0]
            target_q_values = rewards + self.gamma * max_next_q_values * (1 - dones)

        loss = self.criterion(q_values, target_q_values)
        self.losses.append(loss.item())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.step_count += 1
        if self.step_count % self.target_update == 0:
            self.target_net.load_state_dict(self.policy_net.state_dict())
        return loss.item()



In [ ]:
import random

random.seed(666)  # For reproducibility

all_indices = list(range(1, 101))
random.shuffle(all_indices)

# Choose first N for training
train_indices = all_indices[:80]

# Remaining go to testing
test_indices = all_indices[80:100]  # or use random.sample(all_indices[80:], 20) if you want random 20 from rest

print('len train_indices', len(train_indices))
print('train_indices', train_indices)
print('--------------------------------')
print('len test_indices', len(test_indices))
print('test_indices', test_indices)


In [ ]:
# Repeated engine_unit list
repeated_list = train_indices * 3700  # Make 105,000-length list
chunk_size = len(train_indices)
print(chunk_size)
for i in range(0, len(repeated_list), chunk_size):
    chunk = repeated_list[i:i + chunk_size]
    random.shuffle(chunk)
    repeated_list[i:i + chunk_size] = chunk

In [ ]:
print(repeated_list[0:80])
print(repeated_list[80:160])

In [ ]:
episodLength = len(repeated_list)
episodLength

In [153]:
import random
import numpy as np
import matplotlib.pyplot as plt

episode_rewards = []
failure_states = []
episode_losses = []
episode_policy = []
epsilon_min_reached_at = None



def train_dqn(env, agent, episodes=500):
    global epsilon_min_reached_at

    for ep in range(episodes):
        engine_unit = repeated_list[ep]
        state = env.reset(engine_unit)
        total_reward = 0
        done = False
        losses = []

        while not done:
            action = agent.select_action(state)

            # Example: force action=0 if not in scheduled cycle
            if env.cycle not in hierarchical[env.engine_unit - 1]:
                action = 0

            next_state, reward, done, _ = env.step(action)

            agent.store_transition(state, action, reward, next_state, done)
            loss = agent.train_step()

            if loss is not None:
                losses.append(loss)

            state = next_state
            total_reward += reward

        episode_rewards.append(total_reward)
        failure_states.append(env.cycle)
        avg_loss = np.mean(losses) if losses else 0.0
        episode_losses.append(avg_loss)

        if (ep + 1) % 200 == 0:
            agent.epsilon = max(agent.epsilon * agent.epsilon_decay, agent.epsilon_min)

        if epsilon_min_reached_at is None and agent.epsilon < 0.01:
            epsilon_min_reached_at = ep
            agent.epsilon = 0

        # Logging
        print(f"Episode {ep+1}, Total Reward: {total_reward:.4f}, "
              f"Epsilon: {agent.epsilon:.6f}, Avg Loss: {avg_loss:.6f} ")
        print({
            'unit': env.engine_unit,
            'failure_state': env.failure_state,
            'replace_state': env.cycle
        })
        episode_policy.append({
            'unit': env.engine_unit,
            'failure_state': env.failure_state,
            'replace_state': env.cycle,
            'Epsilon': agent.epsilon,
            'avg_loss': int(avg_loss),
        })
        print('---------------------------------------------------------')

    
 


In [154]:
def plot_failure_states(failure_states, engine_unit, epsilons, save_path=None):
    import matplotlib.pyplot as plt

    episodes = list(range(1, len(failure_states) + 1))
    min_threshold = int(hierarchical[engine_unit - 1][0])
    max_threshold = int(hierarchical[engine_unit - 1][-1])

    colors = []
    for f in failure_states:
        if f == max_threshold:
            colors.append('red')
        elif f == max_threshold - 1 or f == max_threshold - 2:
            colors.append('green')
        else:
            colors.append('blue')

    epsilon_min_reached_at = None
    for i, e in enumerate(epsilons):
        if abs(e) < 1e-8:
            epsilon_min_reached_at = i + 1
            break

    plt.figure(figsize=(14, 6))
    plt.scatter(episodes, failure_states, color=colors, s=10, label=f'Failure States (Unit {engine_unit})')

    if epsilon_min_reached_at is not None:
        plt.axvline(x=epsilon_min_reached_at, color='orange', linestyle='--',
                    label=f'ϵ = 0 at Episode {epsilon_min_reached_at}')

    plt.axhline(min_threshold, color='red', linestyle='--', label=f'Min Threshold ({min_threshold})')
    plt.axhline(max_threshold, color='green', linestyle='--', label=f'Max Threshold ({max_threshold})')

    plt.xlabel('Episode')
    plt.ylabel('Failure State (Cycles)')
    plt.title(f'Failure State per Episode - Unit {engine_unit}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()


In [ ]:
episode_rewards = []
failure_states = []
epsilon_min_reached_at = None

env = CustomEnv(is_training=True, verbose=False ,  srla=True)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

agent = DQNAgent(state_dim, action_dim)
train_dqn(env, agent, episodes=episodLength)


In [ ]:
def plot_mean_rewards():
    rewards_per_30 = [np.mean(episode_rewards[i:i+80]) for i in range(0, len(episode_rewards), 80)]
    x = range(1, len(rewards_per_30) + 1)

    plt.figure(figsize=(10, 5))  # عریض‌تر برای دید بهتر
    plt.plot(x, rewards_per_30, color='royalblue', linewidth=2, label='Mean Reward')
    # plt.scatter(x, rewards_per_30, color='orange', s=25, alpha=0.7)  # s=اندازه‌ی نقطه‌ها

    plt.xlabel('Epoch (every 30 episodes)', fontsize=12)
    plt.ylabel('Mean Reward', fontsize=12)
    plt.title('Mean Reward Over Time (every 80 episodes)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.tight_layout()
    plt.show()


plot_mean_rewards()    

In [ ]:
def plot_mean_rewards():
    rewards_per_30 = [np.mean(episode_rewards[i:i+80]) for i in range(0, len(episode_rewards), 80)]
    x = range(1, len(rewards_per_30) + 1)

    plt.figure(figsize=(10, 5))  # عریض‌تر برای دید بهتر
    plt.plot(x, rewards_per_30, color='royalblue', linewidth=2, label='Mean Reward')
    # plt.scatter(x, rewards_per_30, color='orange', s=25, alpha=0.7)  # s=اندازه‌ی نقطه‌ها

    plt.xlabel('Epoch (every 30 episodes)', fontsize=12)
    plt.ylabel('Mean Reward', fontsize=12)
    plt.title('Mean Reward Over Time (every 80 episodes)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.tight_layout()
    plt.show()


plot_mean_rewards()    

In [ ]:
def plot_mean_rewards():
    rewards_per_30 = [np.mean(episode_rewards[i:i+80]) for i in range(0, len(episode_rewards), 80)]
    x = range(1, len(rewards_per_30) + 1)

    plt.figure(figsize=(10, 5))  # عریض‌تر برای دید بهتر
    plt.plot(x, rewards_per_30, color='royalblue', linewidth=2, label='Mean Reward')
    # plt.scatter(x, rewards_per_30, color='orange', s=25, alpha=0.7)  # s=اندازه‌ی نقطه‌ها

    plt.xlabel('Epoch (every 30 episodes)', fontsize=12)
    plt.ylabel('Mean Reward', fontsize=12)
    plt.title('Mean Reward Over Time (every 80 episodes)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.tight_layout()
    plt.show()


plot_mean_rewards()    

save

In [ ]:
def plot_mean_rewards():
    rewards_per_30 = [np.mean(episode_rewards[i:i+80]) for i in range(0, len(episode_rewards), 80)]
    x = range(1, len(rewards_per_30) + 1)

    plt.figure(figsize=(10, 5))
    plt.plot(x, rewards_per_30, color='royalblue', linewidth=2, label='Mean Reward')
    
    plt.xlabel('Epoch (every 30 episodes)', fontsize=12)
    plt.ylabel('Mean Reward', fontsize=12)
    plt.title('Mean Reward Over Time (every 80 episodes)', fontsize=14)
    plt.ylim(-4, -1)  # 👈 This line zooms the y-axis
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.tight_layout()
    plt.show()
plot_mean_rewards()

In [157]:
import os
from collections import defaultdict

# Group failure_states and epsilons by unit
unit_data = defaultdict(lambda: {'failures': [], 'epsilons': []})

for record in episode_policy:
    unit = record['unit']
    unit_data[unit]['failures'].append(record['replace_state'])
    unit_data[unit]['epsilons'].append(record['Epsilon'])

# Create folder if not exists
# output_dir = './results/v-12'
os.makedirs(output_dir, exist_ok=True)

# Plot per unit and save
for unit, data in unit_data.items():
    save_path = os.path.join(output_dir, f'unit_{unit}.png')
    plot_failure_states(
        failure_states=data['failures'],
        engine_unit=unit,
        epsilons=data['epsilons'],
        save_path=save_path
    )


In [ ]:
policy

In [ ]:
for k, v in policy.items():
    v['RUL'] = v['failure_state'] - v['replace_state']

# Optional: pretty print
from pprint import pprint
pprint(policy)

In [ ]:
count_zero_rul = sum(1 for v in policy.values() if v['RUL'] == 0)
print("Number of rows with RUL == 0:", count_zero_rul)


In [ ]:


mean_rul = np.mean([v['RUL'] for v in policy.values()])
print("Mean RUL:", mean_rul)

In [ ]:
test_rewards = []
test_policy = []

# Set env to evaluation mode
test_env = CustomEnv(is_training=False, verbose=False, srla=True)

for unit in test_indices:
    state = test_env.reset(unit)
    done = False
    total_reward = 0

    while not done:
        action = agent.select_action(state)  # Use greedy action (no epsilon)

        # Optional: Force hold if outside danger zone
        if test_env.cycle not in hierarchical[test_env.engine_unit - 1]:
            action = 0

        next_state, reward, done, _ = test_env.step(action)
        state = next_state
        total_reward += reward

    test_rewards.append(total_reward)
    test_policy.append(policy_test[test_env.engine_unit])  # saved inside env.step()

    print(f"Test Unit {unit}, Total Reward: {total_reward:.4f}")
    print(policy_test[test_env.engine_unit])
    print('---------------------------------------------------------')


In [ ]:
df = pd.DataFrame(test_policy)

# Add RUL column
df['RUL'] = df['failure_state'] - df['replace_state']

# Display the table
print(df)

In [ ]:
rul_results = [{'unit': d['unit'], 'RUL': d['failure_state'] - d['replace_state']} for d in test_policy]

rul_results

In [ ]:
mean_rul = sum(r['RUL'] for r in rul_results) / len(rul_results)

print(f"Mean RUL: {mean_rul}")

In [ ]:
count_zero_rul = sum(1 for r in rul_results if r['RUL'] == 0)
print("Number of entries with RUL == 0:", count_zero_rul)
